In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import re 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dropout,Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords

In [ ]:
train = pd.read_csv("/kaggle/input/email-classification-nlp/SMS_train.csv",encoding='latin-1')
test = pd.read_csv("/kaggle/input/email-classification-nlp/SMS_train.csv",encoding='latin-1')

#### Encoding Label

In [ ]:
train['Label'] =  train['Label'].map({"Non-Spam":0,"Spam":1})
test['Label'] =  test['Label'].map({"Non-Spam":0,"Spam":1})

In [ ]:
train.head()

#### Checking missing values

In [ ]:
train.isna().sum()

In [ ]:
train['Label'].value_counts()

#### cleaning

In [ ]:
STOPWORDS = stopwords.words("english")

In [ ]:
def clean(text):
    text = text.lower()
    text = re.sub("(http|https|www)(:|\.)\S+.com"," ",text)
    text = re.sub("[^\w\d]"," ",text)
    text = " ".join([t for t in text.split() if t not in STOPWORDS])
    return text 

In [ ]:
clean("Hello #$$% User !  at Https://gmal.com www.hello.com")

In [ ]:
train['cleaned']=  train['Message_body'].apply(lambda x : clean(x))

In [ ]:
train.head()

In [ ]:
train['cleaned'][:5]

In [ ]:
### Hyperparamters 
vocab_size = 5000
max_len = 30

In [ ]:
token = Tokenizer(num_words=vocab_size)
token.fit_on_texts(train['cleaned'])

x_train = token.texts_to_sequences(train['cleaned'])

In [ ]:
x_test = test['Message_body'].apply(lambda x : clean(x))
x_test = token.texts_to_sequences(x_test)

In [ ]:
x_test =  pad_sequences(x_test,padding='pre',truncating='pre',maxlen=max_len)

In [ ]:
x_train[:5]

In [ ]:
x_train = pad_sequences(x_train,maxlen=max_len,padding='pre',truncating='pre')

In [ ]:
x_train

#### Time to Create  a Neural Network

In [ ]:
model = Sequential([
    Embedding(vocab_size,100,input_length=max_len),
    Dropout(0.2),
    LSTM(200),
    Dropout(0.2),
    Dense(1,activation='sigmoid')
])

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
history =  model.fit(x_train,train['Label'],epochs=5,batch_size=32,validation_data=(x_test,test['Label']))

In [ ]:
history_df = pd.DataFrame(history.history)
history_df[['acc','val_acc']].plot(title='Accuracy')
history_df[['loss','val_loss']].plot(title='Loss')

In [ ]:
loss,acc = model.evaluate(x_test,test['Label'])
acc